<a href="https://colab.research.google.com/github/Nishasathish13/TheSchoolofAI-END3.0/blob/main/Session%205%20-%20TorchText%20%26%20Advanced%20Concepts/Assignment/Session_5_Assignment_dataset3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchtext.datasets import YahooAnswers

In [2]:
help(YahooAnswers)

Help on function YahooAnswers in module torchtext.datasets.yahooanswers:

YahooAnswers(root='.data', split=('train', 'test'))
    YahooAnswers dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 1400000
    
        test: 60000
    
    
    Number of classes
        10
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Can be a string or tuple of strings.
            Default: ('train', 'test')



In [3]:
train_iter = YahooAnswers(split='train')
next(train_iter)

100%|██████████| 319M/319M [00:02<00:00, 145MB/s]


(5,
 "why doesn't an optical mouse work on a glass table? or even on some surfaces? Optical mice use an LED and a camera to rapidly capture images of the surface beneath the mouse.  The infomation from the camera is analyzed by a DSP (Digital Signal Processor) and used to detect imperfections in the underlying surface and determine motion. Some materials, such as glass, mirrors or other very shiny, uniform surfaces interfere with the ability of the DSP to accurately analyze the surface beneath the mouse.  \\nSince glass is transparent and very uniform, the mouse is unable to pick up enough imperfections in the underlying surface to determine motion.  Mirrored surfaces are also a problem, since they constantly reflect back the same image, causing the DSP not to recognize motion properly. When the system is unable to see surface changes associated with movement, the mouse will not work properly.")

In [4]:
# or iterate over a loop

#to display only the first 20 lines
for (ln, (label, line)) in enumerate(train_iter):
  print(ln, label, line)
  if ln == 20:
    break


0 6 What is the best off-road motorcycle trail ? long-distance trail throughout CA i hear that the mojave road is amazing!<br />\nsearch for it online.
1 3 What is Trans Fat? How to reduce that? I heard that tras fat is bad for the body.  Why is that? Where can we find it in our daily food? Trans fats occur in manufactured foods during the process of partial hydrogenation, when hydrogen gas is bubbled through vegetable oil to increase shelf life and stabilize the original polyunsatured oil. The resulting fat is similar to saturated fat, which raises "bad" LDL cholesterol and can lead to clogged arteries and heart disease. \nUntil very recently, food labels were not required to list trans fats, and this health risk remained hidden to consumers. In early July, FDA regulations changed, and food labels will soon begin identifying trans fat content in processed foods.
2 7 How many planes Fedex has? I heard that it is the largest airline in the world according to the www.fedex.com web site:\

In [5]:
# dataloader
from torch.utils.data import DataLoader
train_iter = YahooAnswers(split='train')
dataloader = DataLoader(train_iter, batch_size=16, shuffle=False)

next(iter(dataloader))
#displays 8 labels as batch size = 8

[tensor([5, 6, 3, 7, 7, 5, 2, 8, 5, 2, 4, 4, 7, 5, 9, 2]),
 ("why doesn't an optical mouse work on a glass table? or even on some surfaces? Optical mice use an LED and a camera to rapidly capture images of the surface beneath the mouse.  The infomation from the camera is analyzed by a DSP (Digital Signal Processor) and used to detect imperfections in the underlying surface and determine motion. Some materials, such as glass, mirrors or other very shiny, uniform surfaces interfere with the ability of the DSP to accurately analyze the surface beneath the mouse.  \\nSince glass is transparent and very uniform, the mouse is unable to pick up enough imperfections in the underlying surface to determine motion.  Mirrored surfaces are also a problem, since they constantly reflect back the same image, causing the DSP not to recognize motion properly. When the system is unable to see surface changes associated with movement, the mouse will not work properly.",
  'What is the best off-road motorc

In [ ]:
help(DataLoader)

Torchtext has revamped the very basic components of the torchtext library, including vocab, word vectors, tokenizer. These are the basic data processing building blocks for a raw text strings. 

Here is an example for typical NLP data processing with tokenizer and vocabulary. The first step is to build a vocabulary with the raw training dataset. Here we use built in factory function build_vocab_from_iterator which accepts iterator that yield list or iterator of tokens. Users can also pass any special symbols to be added to the vocabulary.

In [6]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = YahooAnswers(split='train')

#to feed in sentences one by one
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

#sets the default index = 0 for the unknown words, i.e., words not in the vocabulary
vocab.set_default_index(vocab["<unk>"])

In [7]:
def st(*transforms):
  def func(txt_io):
    for transform in transforms:
      txt_io = transform(txt_io)
    return txt_io
  return func

In [ ]:
#theschoolofai is the unknown word in our vocab, hence index set to 0
vocab(['here', 'is', 'an', 'example', 'theschoolofai'])

[51, 14, 68, 2030, 0]

Prepare the text processing pipeline with the tokenizer and vocabulary. The text and label pipelines will be used to process the raw data strings from the dataset iterators.

In [8]:
#we need to pass our sentences through the tokenizer first to normalise it before sending it the vocab
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)-1

In [9]:
text_pipeline('here is the an example')

[150, 11, 2, 55, 354]

In [10]:
label_pipeline('10')

9

## Generate data batch and iterator

torch.utils.data.DataLoader is recommended for PyTorch users. It works with a map-style dataset that implements the getitem() and len() protocols, and represents a map from indices/keys to data samples. It also works with an iterable dataset with the shuffle argument of False.

Before sending to the model, collate_fn function works on a batch of samples generated from DataLoader. The input to collate_fn is a batch of data with the batch size in DataLoader, and collate_fn processes them according to the data processing pipelines declared previously. Pay attention here and make sure that collate_fn is declared as a top level def. This ensures that the function is available in each worker.

In [11]:
from torch.utils.data import DataLoader
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

EmbeddingBag is a useful feature to consume sparse ids and produce embeddings. Later we are going to see examples where the text entries in the original batch input will be backed into a list and concatenated as a single tensor for the input of nn.EmbeddingBag.

It is doing two things. The first step is to create an embedding and the second step is to reduce (sum/mean/max, according to the "mode" argument) the embedding output across dimension 0. So this is equivalent to torch.nn.functional.embedding, followed by torch.sum/mean/max. However, the conceptual two step process does not reflect how it's actually implemented. Since embedding_bag does not need to return the intermediate result, it doesn't actually generate a Tensor object for the embedding. It just goes straight to computing the reduction, pulling in the appropriate data from the weight argument according to the indices in the input argument. Avoiding the creation of the embedding Tensor allows for better performance

In [ ]:
weight = torch.randn(3, 4)
weight

tensor([[ 0.1543, -1.5977, -0.2257,  0.1658],
        [-0.3355, -2.6362,  0.4767, -0.1704],
        [ 2.6389,  0.8871, -1.5911,  0.5172]])

In [ ]:
indices = torch.tensor([2, 1])
indices

tensor([2, 1])

In [ ]:
embedding_dwork = torch.nn.functional.embedding(indices, weight)
embedding_dwork

tensor([[ 2.6389,  0.8871, -1.5911,  0.5172],
        [-0.3355, -2.6362,  0.4767, -0.1704]])

In [ ]:
embedding_dwork_mean = embedding_dwork.mean(dim=0, keepdim=True)
embedding_dwork_mean

tensor([[ 1.1517, -0.8746, -0.5572,  0.1734]])

In [ ]:
embedding_bag = torch.nn.functional.embedding_bag(indices, weight, torch.tensor([0]), mode='mean')
embedding_bag

tensor([[ 1.1517, -0.8746, -0.5572,  0.1734]])

In this example, the text entries in the original data batch input are packed into a list and concatenated as a single tensor for the input of nn.EmbeddingBag. The offset is a tensor of delimiters to represent the beginning index of the individual sequence in the text tensor. Label is a tensor saving the labels of individual text entries.

In [12]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label)) #appending the labels to label_list
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text) 
         offsets.append(processed_text.size(0)) #appending the length of the sentence
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)    

In [13]:
train_iter = YahooAnswers(split='train')
dataloader = DataLoader(train_iter, batch_size=16, shuffle=False, collate_fn=collate_batch)

The model is composed of the nn.EmbeddingBag <https://pytorch.org/docs/stable/nn.html?highlight=embeddingbag#torch.nn.EmbeddingBag>__ layer plus a linear layer for the classification purpose. nn.EmbeddingBag with the default mode of "mean" computes the mean value of a “bag” of embeddings. Although the text entries here have different lengths, nn.EmbeddingBag module requires no padding here since the text lengths are saved in offsets.

Additionally, since nn.EmbeddingBag accumulates the average across the embeddings on the fly, nn.EmbeddingBag can enhance the performance and memory efficiency to process a sequence of tensors.

In [14]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

We build a model with the embedding dimension of 64. The vocab size is equal to the length of the vocabulary instance. The number of classes is equal to the number of labels,

In [15]:
train_iter = YahooAnswers(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [16]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

Split the dataset and run the model
Since the original AG_NEWS has no valid dataset, we split the training dataset into train/valid sets with a split ratio of 0.95 (train) and 0.05 (valid). Here we use torch.utils.data.dataset.random_split <https://pytorch.org/docs/stable/data.html?highlight=random_split#torch.utils.data.random_split>__ function in PyTorch core library.

CrossEntropyLoss <https://pytorch.org/docs/stable/nn.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss>__ criterion combines nn.LogSoftmax() and nn.NLLLoss() in a single class. It is useful when training a classification problem with C classes. SGD <https://pytorch.org/docs/stable/_modules/torch/optim/sgd.html>__ implements stochastic gradient descent method as the optimizer. The initial learning rate is set to 5.0. StepLR <https://pytorch.org/docs/master/_modules/torch/optim/lr_scheduler.html#StepLR>__ is used here to adjust the learning rate through epochs.

In [17]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = YahooAnswers()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=False, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/20782 batches | accuracy    0.285
| epoch   1 |  1000/20782 batches | accuracy    0.437
| epoch   1 |  1500/20782 batches | accuracy    0.514
| epoch   1 |  2000/20782 batches | accuracy    0.551
| epoch   1 |  2500/20782 batches | accuracy    0.575
| epoch   1 |  3000/20782 batches | accuracy    0.593
| epoch   1 |  3500/20782 batches | accuracy    0.608
| epoch   1 |  4000/20782 batches | accuracy    0.615
| epoch   1 |  4500/20782 batches | accuracy    0.624
| epoch   1 |  5000/20782 batches | accuracy    0.629
| epoch   1 |  5500/20782 batches | accuracy    0.634
| epoch   1 |  6000/20782 batches | accuracy    0.642
| epoch   1 |  6500/20782 batches | accuracy    0.644
| epoch   1 |  7000/20782 batches | accuracy    0.650
| epoch   1 |  7500/20782 batches | accuracy    0.652
| epoch   1 |  8000/20782 batches | accuracy    0.654
| epoch   1 |  8500/20782 batches | accuracy    0.654
| epoch   1 |  9000/20782 batches | accuracy    0.650
| epoch   1 |  9500/20782 ba

In [18]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.705
